# Average antibody escape across multiple fit `polyclonal` models

Get parameterized variables from [papermill](https://papermill.readthedocs.io/):

In [ ]:
# papermill parameters cell (tagged as `parameters`)
antibody = None
escape_avg_method = None
avg_pickle = None
selection_groups = None

In [ ]:
# This is just a temporary completed parameters cell like papermill will make when called with antibody REGN10933
selection_groups = {
    "LibA_2022-02-10a_thaw-3_REGN10933_1": {
        "date": "2022-02-10a",
        "library": "LibA",
        "replicate": 1,
        "virus_batch": "thaw-3",
    },
    "LibA_2022-02-10a_thaw-3_REGN10933_2": {
        "date": "2022-02-10a",
        "library": "LibA",
        "replicate": 2,
        "virus_batch": "thaw-3",
    },
    "LibB_2022-03-02_thaw-3_REGN10933_1": {
        "date": "2022-03-02",
        "library": "LibB",
        "replicate": 1,
        "virus_batch": "thaw-3",
    },
}
antibody = "REGN10933"
escape_avg_method = "median"
avg_pickle = "results/antibody_escape/REGN10933.pickle"

In [ ]:
print(escape_avg_method)

In [ ]:
print(avg_pickle)

In [ ]:
print(selection_groups)